### MONSIEUR CELUI LA  AUSSI EST LE PREMIER CODE FAIT AVEC UN PREMIER TYPE DEMBEDDING QUI EST LE BERT EMBEDDING , MAIS CELUI LA EST POUR LA CLASSIFICATION TYPE LOSS-OF-INFO-CONTENT


In [103]:
import pandas as pd

data=pd.read_csv("fusionned_file.csv",sep=";")
data.head()


,source sentence,simplified sentence,run_id,snt_id,No error,Random generation,Syntax error,Contradiction,Simple punctuation / grammar errors,Redundancy,Format misalignement,Prompt misalignement,Out-of-Scope Generation,Topic shift,Oversimplification of Logical Arguments,Overgeneralization,Loss of Informative Content,Factuality hallucination,Faithfulness hallucination,Commentaire
0,":[0],""non-coding RNAs (lncRNAs) may play crit...","Long non-coding RNAs, or lncRNAs, may play si...",UBO_Task3.1_Phi4mini-sl,G04.2_2519200480_1,VRAI,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN
1,"Recently, lncRNAs have been widely identified ...",Small RNA molecules are found in complex cell...,UBO_Task3.1_Phi4mini-sl,G04.2_2519200480_2,VRAI,False,False,False,False,False,False,False,False,False,False,False,False,False,False,NaN
2,"However, there is rare information about them ...",There is unusual information here that doesn'...,UBO_Task3.1_Phi4mini-sl,G04.2_2519200480_3,FAUX,False,False,False,False,False,False,False,True,True,False,False,False,False,False,NaN
3,"During humanitarian crises, there is a need fo...","When crises happen, people need quick and larg...",UBO_Task3.1_Phi4mini-sl,G07.1_3007380484_1,FAUX,False,False,False,False,False,False,False,True,False,False,False,False,False,False,NaN
4,Such need creates the base for misinformation ...,This reduction helps in combating climate chan...,UBO_Task3.1_Phi4mini-sl,G07.1_3007380484_2,FAUX,False,False,False,False,True,False,True,False,True,False,False,False,False,False,NaN


In [104]:
data.drop(columns=['run_id', 'snt_id','No error','Syntax error','Out-of-Scope Generation','Random generation','Simple punctuation / grammar errors','Redundancy','Contradiction','Format misalignement','Prompt misalignement','Topic shift',	'Oversimplification of Logical Arguments',	'Overgeneralization','Factuality hallucination',	'Faithfulness hallucination',	'Commentaire'], inplace=True)

In [105]:
data.head(1)

,source sentence,simplified sentence,Loss of Informative Content
0,":[0],""non-coding RNAs (lncRNAs) may play crit...","Long non-coding RNAs, or lncRNAs, may play si...",False


In [106]:
bool_columns = data.columns[data.dtypes == 'bool']

# Calculer les pourcentages de TRUE et FALSE pour chaque colonne booléenne
percentages = {
    col: {
        'TRUE (%)': data[col].mean() * 100,
        'FALSE (%)': (1 - data[col].mean()) * 100
    }
    for col in bool_columns
}

# Convertir en DataFrame pour afficher les résultats
result_df = pd.DataFrame(percentages).T

# Afficher les résultats
#import ace_tools as tools; tools.display_dataframe_to_user(name="Pourcentage des Colonnes Booléennes", dataframe=result_df)
print(result_df)

                             FALSE (%)   TRUE (%)
Loss of Informative Content  79.393939  20.606061


## Preprocessing before embeddings

In [107]:
import pandas as pd
import re

# Fonction de nettoyage
def clean_text(text):
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)  # Supprime caractères spéciaux (, . () [])
    text = text.lower().strip()  # Convertit en minuscules et supprime les espaces inutiles
    return text

data['source sentence'] = data['source sentence'].apply(clean_text)
data['simplified sentence'] = data['simplified sentence'].apply(clean_text)
print(data.head())


                                     source sentence  \
0  0noncoding rnas lncrnas may play critical role...   
1  recently lncrnas have been widely identified a...   
2  however there is rare information about them i...   
3  during humanitarian crises there is a need for...   
4  such need creates the base for misinformation ...   

                                 simplified sentence  \
0  long noncoding rnas or lncrnas may play signif...   
1  small rna molecules are found in complex cells...   
2  there is unusual information here that doesnt ...   
3  when crises happen people need quick and large...   
4  this reduction helps in combating climate chan...   

   Loss of Informative Content  
0                        False  
1                        False  
2                        False  
3                        False  
4                        False  


### Applying  Embeddings : SENTENCE-BERT

- Sentence-BERT (SBERT) est utilisé pour générer des embeddings qui capturent le sens global des phrases, idéal pour comparer la source et la simplification.

- Le modèle all-MiniLM-L6-v2 est rapide, léger et bien adapté pour détecter des erreurs comme Loss of Informative Content en évaluant la similarité sémantique.

- Embeddings permettent de calculer des métriques (similarité cosinus, différence vectorielle) pour détecter si des informations importantes ont été perdues.

In [108]:
from sentence_transformers import SentenceTransformer
import pandas as pd
from sklearn.decomposition import PCA
import numpy as np

# Charger le modèle Sentence-BERT
model = SentenceTransformer('all-MiniLM-L6-v2')

# Générer les embeddings pour chaque colonne
data['source_embedding'] = data['source sentence'].apply(model.encode)
data['simplified_embedding'] = data['simplified sentence'].apply(model.encode)

# Conversion des embeddings en matrice pour réduction de dimension
source_embeddings = np.vstack(data['source_embedding'].values)
simplified_embeddings = np.vstack(data['simplified_embedding'].values)


In [109]:
# Réduction de dimension à 50 via PCA
pca = PCA(n_components=50, random_state=42)
source_latent = pca.fit_transform(source_embeddings)
simplified_latent = pca.transform(simplified_embeddings)

# Ajouter les vecteurs réduits au DataFrame
data['source_latent'] = list(source_latent)
data['simplified_latent'] = list(simplified_latent)


# Fonction pour calculer la similarité cosinus 

In [110]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import euclidean

def compare_embeddings(vec1, vec2, metric='cosine'):
    if metric == 'cosine':
        return cosine_similarity([vec1], [vec2])[0][0]
    elif metric == 'euclidean':
        return euclidean(vec1, vec2)

# Ajouter une colonne de similarité au DataFrame
data['similarity'] = data.apply(
    lambda row: compare_embeddings(row['source_latent'], row['simplified_latent'], metric='cosine'), axis=1
)

# Afficher le DataFrame final
print(data[['source sentence', 'simplified sentence', 'similarity']])


                                       source sentence  \
0    0noncoding rnas lncrnas may play critical role...   
1    recently lncrnas have been widely identified a...   
2    however there is rare information about them i...   
3    during humanitarian crises there is a need for...   
4    such need creates the base for misinformation ...   
..                                                 ...   
325  the hmm predicts the users motion and computes...   
326  our approach then selects effective search loc...   
327  the selection criterion hereby considers the v...   
328  we performed both realworld and extensive simu...   
329  in comparison to a greedy maximum coverage app...   

                                   simplified sentence  similarity  
0    long noncoding rnas or lncrnas may play signif...    0.937647  
1    small rna molecules are found in complex cells...    0.808469  
2    there is unusual information here that doesnt ...    0.697989  
3    when crises happen peo

In [111]:
data.head(-10)

,source sentence,simplified sentence,Loss of Informative Content,source_embedding,simplified_embedding,source_latent,simplified_latent,similarity
0,0noncoding rnas lncrnas may play critical role...,long noncoding rnas or lncrnas may play signif...,False,"[0.02420081, -0.07566879, -0.020376744, -0.045...","[-0.0067747403, -0.099808805, 0.03531126, 0.00...","[-0.1402851, 0.3993774, 0.016698837, 0.2754986...","[-0.13780719, 0.41999602, 0.01167783, 0.178029...",0.937647
1,recently lncrnas have been widely identified a...,small rna molecules are found in complex cells...,False,"[-0.012879153, -0.030902008, -0.023145385, -0....","[0.06118244, 0.017324567, -0.009085979, -0.018...","[-0.11340841, 0.25922093, 0.027499028, 0.28721...","[-0.1848618, 0.28758222, 0.018048752, 0.236620...",0.808469
2,however there is rare information about them i...,there is unusual information here that doesnt ...,False,"[0.0074299593, 0.0038072907, 0.0026281097, 0.0...","[-0.030303897, 0.0016395211, -0.056994755, 0.0...","[-0.06470882, 0.09986454, 0.031302515, 0.05054...","[0.10654827, -0.021748746, -0.02932733, 0.1406...",0.697989
3,during humanitarian crises there is a need for...,when crises happen people need quick and large...,False,"[0.048965454, 0.03705088, 0.03230201, 0.007998...","[0.007581319, 0.04263264, 0.017020613, 0.01053...","[0.17458947, -0.074454896, -0.053535562, 0.001...","[0.26496148, -0.17702724, 0.18001825, 0.138153...",0.574447
4,such need creates the base for misinformation ...,this reduction helps in combating climate chan...,False,"[-0.011502491, -0.052530874, 0.030595861, 0.04...","[0.014214046, 0.090169616, 0.049768943, 0.0073...","[0.5497599, 0.13537794, -0.20753784, -0.006698...","[0.06093421, 0.017855573, -0.042385153, -0.054...",0.171497
...,...,...,...,...,...,...,...,...
315,this makes it a particularly useful tool for t...,it helps us plan a path from start to finish,False,"[-0.062028, -0.016926745, -0.06980939, -0.0269...","[0.014475317, -0.0047829207, 0.042760096, 0.05...","[-0.33877483, -0.11205372, -0.11344484, -0.126...","[-0.15736, 0.06647777, -0.12156913, -0.1263196...",0.753362
316,by means of time synchronization the model ext...,things happen at the same time in many directions,True,"[0.021785203, -0.07796893, 0.013258484, 0.0140...","[0.023500811, -0.073400564, 0.049324512, 0.033...","[-0.2969103, 0.15516447, 0.14351404, -0.283581...","[-0.032525104, 0.27469513, -0.04565381, -0.273...",0.481472
317,the framework excels in terms of execution tim...,the special tool can do tasks quickly even wit...,True,"[0.03526398, -0.057930723, -0.080005206, -0.02...","[-0.036649317, 0.005778734, -0.025373813, 0.02...","[-0.4152019, 0.07182891, 0.023735616, -0.18246...","[-0.37105012, 0.017789314, -0.08459009, -0.076...",0.657473
318,we demonstrate our method in two different set...,we show how our special way of doing things is...,False,"[0.031496752, 0.015216269, 0.007014798, -0.032...","[0.04590474, 0.008630812, 0.04923024, 0.026617...","[-0.47688702, 0.18691579, -0.019883687, -0.192...","[-0.09202174, 0.030468311, -0.18715963, -0.188...",0.435370


In [112]:
data.drop(columns=['source sentence','simplified sentence','source_embedding','simplified_embedding'], inplace=True)

In [113]:
data.head()

,Loss of Informative Content,source_latent,simplified_latent,similarity
0,False,"[-0.1402851, 0.3993774, 0.016698837, 0.2754986...","[-0.13780719, 0.41999602, 0.01167783, 0.178029...",0.937647
1,False,"[-0.11340841, 0.25922093, 0.027499028, 0.28721...","[-0.1848618, 0.28758222, 0.018048752, 0.236620...",0.808469
2,False,"[-0.06470882, 0.09986454, 0.031302515, 0.05054...","[0.10654827, -0.021748746, -0.02932733, 0.1406...",0.697989
3,False,"[0.17458947, -0.074454896, -0.053535562, 0.001...","[0.26496148, -0.17702724, 0.18001825, 0.138153...",0.574447
4,False,"[0.5497599, 0.13537794, -0.20753784, -0.006698...","[0.06093421, 0.017855573, -0.042385153, -0.054...",0.171497


### Importing Packages!!

In [114]:
import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import balanced_accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC

### SPLIT

In [115]:
X = data[['source_latent', 'simplified_latent', 'similarity']]
y = data['Loss of Informative Content'].astype(int)

# Combiner les colonnes d'embeddings en un vecteur unique
def combine_features(row):
    return np.concatenate([row['source_latent'], row['simplified_latent'], [row['similarity']]])

# Appliquer la combinaison sur toutes les lignes
X_combined = np.array(X.apply(combine_features, axis=1).tolist())

# Diviser en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)


## SMOTE OVERSAMPLING CLASSE MINORITAIRE

In [116]:
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)


## Logistic Regression

In [117]:
logistic_model = LogisticRegression(max_iter=1000, random_state=42)
logistic_model.fit(X_train_resampled, y_train_resampled)
y_pred = logistic_model.predict(X_test)


In [118]:
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=["False", "True"]))


Confusion Matrix:
[[35 16]
 [ 4 11]]

Classification Report:
              precision    recall  f1-score   support

       False       0.90      0.69      0.78        51
        True       0.41      0.73      0.52        15

    accuracy                           0.70        66
   macro avg       0.65      0.71      0.65        66
weighted avg       0.79      0.70      0.72        66



In [119]:
print(f"The balanced accuracy of the model is "
      f"{balanced_accuracy_score(y_test, y_pred):.3f}")

The balanced accuracy of the model is 0.710


### Modeling with svm

In [120]:
svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train_resampled, y_train_resampled)
y_pred = svm_model.predict(X_test)


In [121]:
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=["False", "True"]))

Confusion Matrix:
[[34 17]
 [ 4 11]]

Classification Report:
              precision    recall  f1-score   support

       False       0.89      0.67      0.76        51
        True       0.39      0.73      0.51        15

    accuracy                           0.68        66
   macro avg       0.64      0.70      0.64        66
weighted avg       0.78      0.68      0.71        66



In [122]:
print(f"The balanced accuracy of the model is "
      f"{balanced_accuracy_score(y_test, y_pred):.3f}")

The balanced accuracy of the model is 0.700
